# Data Loading & Cleaning (Fixed)

This notebook addresses the data quality issues found in `train_raw.csv`.
The key steps are:
1.  **Load Data**: Read the raw CSV file.
2.  **Initial Cleanup**: Fix column names and apply initial data types.
3.  **Handle Categorical Features**: Clean up and encode categorical columns like `occupation`, `credit_mix`, and `payment_of_min_amount`.
4.  **Handle Missing Values (NaNs)**: Systematically impute missing values for both numeric and categorical columns.
5.  **Handle Outliers**: Cap extreme values in numeric columns to reduce their skewness.
6.  **Feature Transformation**: Convert columns like `credit_history_age` to a numeric format.
7.  **Drop Unnecessary Columns**: Remove identifiers that are not useful for modeling.
8.  **Save Cleaned Data**: Export the processed DataFrame to a new CSV file.

In [40]:
import pandas as pd
import numpy as np
import re

## 1. Load Data and Initial Schema Application

In [41]:
file_name = "train_raw.csv"

# Define the schema for data types
string_cols = [
    "customer_id", "month", "name", "ssn",
    "occupation", "type_of_loan", "credit_history_age",
    "credit_mix", "payment_of_min_amount", "payment_behaviour", "credit_score"
]

# Read raw CSV without strict dtype enforcement for numeric columns
df = pd.read_csv(file_name, na_values=["", "NA", "N/A", "NaN", "_______", "_", "!@9#%8"])

# Convert column names to lower case
df.columns = df.columns.str.lower()

# Coerce numeric columns, turning errors into NaN
for col in df.columns:
    if col not in string_cols:
        # .str.replace(',', '') - converts numbers 1,200 to 1200
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors="coerce")

# Create a unique row_id and drop original identifiers
if "id" in df.columns and "customer_id" in df.columns:
    df["row_id"] = df.index
    df = df.drop(columns=["id", "customer_id", "ssn", "name"], errors="ignore")

df.head()

/tmp/ipykernel_70742/1559141255.py:11: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name, na_values=["", "NA", "N/A", "NaN", "_______", "_", "!@9#%8"])


,month,age,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,type_of_loan,...,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score,row_id
0,January,23.0,Scientist,19114.12,1824.843333,3,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.415295,High_spent_Small_value_payments,312.494089,Good,0
1,February,23.0,Scientist,19114.12,NaN,3,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,809.98,31.944960,NaN,No,49.574949,118.280222,Low_spent_Large_value_payments,284.629162,Good,1
2,March,-500.0,Scientist,19114.12,NaN,3,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521,Low_spent_Medium_value_payments,331.209863,Good,2
3,April,23.0,Scientist,19114.12,NaN,3,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.458074,Low_spent_Small_value_payments,223.451310,Good,3
4,May,23.0,Scientist,19114.12,1824.843333,3,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153,High_spent_Medium_value_payments,341.489231,Good,4


## 2. Clean Categorical and String Columns

In [42]:
# Clean 'occupation'
df['occupation'] = df['occupation'].replace('_______', 'Unknown')
df['occupation'] = df['occupation'].fillna('Unknown')

# Clean 'credit_mix'
df['credit_mix'] = df['credit_mix'].replace('_', 'Unknown')
df['credit_mix'] = df['credit_mix'].fillna('Unknown')

# Clean 'payment_of_min_amount'
df['payment_of_min_amount'] = df['payment_of_min_amount'].replace('NM', pd.NA)
# Impute with mode
mode_payment = df['payment_of_min_amount'].mode()[0]
df['payment_of_min_amount'] = df['payment_of_min_amount'].fillna(mode_payment)

# Clean 'payment_behaviour'
df['payment_behaviour'] = df['payment_behaviour'].replace('!@9#%8', pd.NA)
mode_behaviour = df['payment_behaviour'].mode()[0]
df['payment_behaviour'] = df['payment_behaviour'].fillna(mode_behaviour)

# One-hot encode 'credit_mix' and 'payment_of_min_amount'
df = pd.get_dummies(df, columns=['credit_mix', 'payment_of_min_amount'], prefix=['credit_mix', 'payment_min'], dummy_na=False)

# One-hot encode 'payment_behaviour'
df = pd.get_dummies(df, columns=['payment_behaviour'], prefix=['pay_beh'], dummy_na=False)

# One-hot encode 'occupation'
df = pd.get_dummies(df, columns=['occupation'], prefix=['occ'], dummy_na=False)

## 3. Clean and Transform `type_of_loan`

In [43]:
# Extract unique loan types
loan_types = set()
for val in df["type_of_loan"].dropna():
    normalized = val.replace(" and", ", ")
    types = [t.strip() for t in normalized.split(",") if t.strip()]
    loan_types.update(types)

# Build all new columns as dict
loan_data = {}
for loan in sorted(loan_types):
    column_name = f"has_{loan.lower().replace(' ', '_').replace('-', '_')}"
    loan_data[column_name] = df["type_of_loan"].apply(
        lambda s, loan=loan: 1 if pd.notna(s) and loan in s else 0
    ).astype("int8")

# Create DataFrame once
loan_dummies = pd.DataFrame(loan_data, index=df.index)

# Merge and clean
df = pd.concat([df.drop(columns=["type_of_loan"], errors="ignore"), loan_dummies], axis=1).copy()

## 4. Transform `credit_history_age`

In [44]:
from typing import Optional


def convert_string_to_months(s: str) -> Optional[int]:
    if pd.isna(s):
        return pd.NA
    try:
        # Matches "X Years and Y Months"
        match = re.search(r"(\d+)\s*Years\s*and\s*(\d+)\s*Months", s)
        if match:
            years = int(match.group(1))
            months = int(match.group(2))
            return years * 12 + months
    except (ValueError, AttributeError):
        return pd.NA
    return pd.NA

df["credit_history_months"] = df["credit_history_age"].apply(convert_string_to_months).astype('Int64')
median_history_months = df["credit_history_months"].median()
df["credit_history_months"] = df["credit_history_months"].fillna(median_history_months)
df = df.drop(columns=["credit_history_age"], errors="ignore")

## 5. Handle Outliers and Impute Missing Numeric Values

Here we will address the numeric columns with outliers and missing values.
The strategy is:
1.  **Clip/Cap**: For columns with extreme outliers, we cap the values at a reasonable percentile (e.g., 99th) or a logical maximum.
2.  **Impute**: We fill missing values (`NaN`) using the median, which is robust to outliers.

In [45]:
numeric_cols_to_process = {
    'annual_income': (0, 0.99),
    'monthly_inhand_salary': (0, 0.99),
    'num_bank_accounts': (0, 25), # Cap at a reasonable number
    'num_credit_card': (0, 25),   # Cap at a reasonable number
    'num_of_loan': (0, 25),        # Cap at a reasonable number
    'num_credit_inquiries': (0, 0.99),
    'changed_credit_limit': (None, 0.99), # Allow negative, cap upper
    'outstanding_debt': (0, None),
    'amount_invested_monthly': (0, 0.99),
    'total_emi_per_month': (0, 0.99),
    'age': (18, 70), # Clip to a reasonable age range
    'num_of_delayed_payment': (0, 0.99),
    'delay_from_due_date': (None, None), # No obvious outliers to cap
    'interest_rate': (0, 0.99), # Cap extreme outliers
    'monthly_balance': (None, None) # Impute NaNs
}

for col, (lower_bound, upper_bound) in numeric_cols_to_process.items():
    if col in df.columns:
        # Cap outliers
        if lower_bound is not None:
            if isinstance(lower_bound, float):
                lower_quantile = df[col].quantile(lower_bound)
                df[col] = df[col].clip(lower=lower_quantile)
            else:
                df[col] = df[col].clip(lower=lower_bound)

        if upper_bound is not None:
            if isinstance(upper_bound, float):
                upper_quantile = df[col].quantile(upper_bound)
                df[col] = df[col].clip(upper=upper_quantile)
            else:
                 df[col] = df[col].clip(upper=upper_bound)

        # Impute missing values with the median
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

# Convert integer-like columns to int
for col in ['num_bank_accounts', 'num_credit_card', 'num_of_loan', 'num_of_delayed_payment', 'age']:
    if col in df.columns:
        df[col] = df[col].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 57 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   month                                     100000 non-null  object 
 1   age                                       100000 non-null  int64  
 2   annual_income                             100000 non-null  float64
 3   monthly_inhand_salary                     100000 non-null  float64
 4   num_bank_accounts                         100000 non-null  int64  
 5   num_credit_card                           100000 non-null  int64  
 6   interest_rate                             100000 non-null  float64
 7   num_of_loan                               100000 non-null  int64  
 8   delay_from_due_date                       100000 non-null  int64  
 9   num_of_delayed_payment                    100000 non-null  int64  
 10  changed_credit_limit 

## 6. Final Review and Save

In [46]:
# Display summary statistics for the cleaned data
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,100000.0,33.9587,11.346718,18.0,25.0,33.0,42.0,70.0
annual_income,100000.0,53655.449822,55828.629917,7005.93,20062.86,37550.74,70064.92,483847.78
monthly_inhand_salary,100000.0,4023.638416,2942.531175,303.645417,1792.084167,3093.745,5371.525,13814.93
num_bank_accounts,100000.0,5.62593,3.411237,0.0,3.0,6.0,7.0,25.0
num_credit_card,100000.0,5.97462,3.544746,0.0,4.0,5.0,7.0,25.0
interest_rate,100000.0,57.92005,328.219315,1.0,8.0,13.0,20.0,2865.01
num_of_loan,100000.0,3.46733,2.826389,0.0,2.0,3.0,5.0,25.0
delay_from_due_date,100000.0,21.06878,14.860104,-5.0,10.0,18.0,28.0,67.0
num_of_delayed_payment,100000.0,13.49315,6.057374,0.0,9.0,14.0,18.0,27.0
changed_credit_limit,100000.0,10.355767,6.682105,-6.49,5.42,9.4,14.66,28.81


In [47]:
# Drop the unique identifier before saving
df = df.drop(columns=['row_id'], errors='ignore')

# Save the cleaned dataframe
df.to_csv("train_cleaned.csv", index=False)

print("Cleaned data saved to train_cleaned.csv")

Cleaned data saved to train_cleaned.csv
